In [1]:
import os
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark import SQLContext
import pyspark

In [2]:
num_nodes = 10 # one of the nodes is reserved for master, thus cluster 11 have 10 available nodes
total_vcores = 120 # get it from SparkUI
total_memory = 520 # get it from SparkUI, in GB
num_executor_per_node = 4 # set this manually, number of concurrent jobs on a single node

num_vcores_per_node = total_vcores/num_nodes
memory_per_node = total_memory/num_nodes
total_executors = num_nodes*num_executor_per_node

In [3]:
memory_percent_required = 0.3 # ranges from [0-1]
vcores_percent_required = 0.4 # ranges from [0-1]

In [4]:
spark_executor_cores = 2 # number of vcores for each executor
spark_executor_instances = int(vcores_percent_required*total_vcores) # instance are translated to vcores on SparkUI
#spark.executor.memory + spark.yarn.executor.memoryOverhead = memory per node / number of executors per node
#spark_executor_memory = int(memory_percent_required * 1000*(memory_per_node/num_executor_per_node)/1.10)
spark_executor_memory = int(1000*((memory_percent_required * total_memory)/(spark_executor_instances))/1.10)
spark_yarn_executor_memoryOverhead = int(spark_executor_memory * 0.10)
spark_default_parallelism = 4 * spark_executor_instances * spark_executor_cores
print('spark.executor.cores', spark_executor_cores)
print('spark.executor.instances', spark_executor_instances)
print('spark.executor.memory (MB)', spark_executor_memory)
print('spark.yarn.executor.memoryOverhead (MB)', spark_yarn_executor_memoryOverhead)
print('spark.default.parallelism', spark_default_parallelism)

spark.executor.cores 2
spark.executor.instances 48
spark.executor.memory (MB) 2954
spark.yarn.executor.memoryOverhead (MB) 295
spark.default.parallelism 384


In [5]:
spark_cluster='hadoop-green11'
os.environ['SPARK_HOME'] = "/usr/hdp/3.1.4.0-315/spark2"
os.environ['PYTHONPATH'] = "/usr/hdp/3.1.4.0-315/spark2/python/:$PYTHONPATH"
os.environ['PYTHONPATH'] = "/usr/hdp/3.1.4.0-315/spark2/python/lib/py4j-0.10.7-src.zip:$PYTHONPATH"
os.environ['HDP_VERSION'] = "3.1.4.0-315"
os.environ['JAVA_HOME'] = "/usr/jdk64/jdk1.8.0_112/"
os.environ['PYSPARK_PYTHON'] = "/u01/anaconda3/bin/python"
os.environ['HADOOP_CONF_DIR'] = "/usr/hdp/3.1.4.0-315/hadoop-yarn/etc/{}".format(spark_cluster)

spark_conf = SparkConf().setAppName('Cluster-Capacity-Test').setAll([
    ('spark.master', 'yarn'),
    ("spark.executor.cores", spark_executor_cores),
    ("spark.executor.instances", spark_executor_instances),
    ("spark.executor.memory", '{}mb'.format(spark_executor_memory)),
    ("spark.yarn.executor.memoryOverhead", '{}mb'.format(spark_yarn_executor_memoryOverhead)),
    ("spark.default.parallelism", spark_default_parallelism),
    #("spark.sql.shuffle.partitions", 200),
    #("spark.serializer", "org.apache.spark.serializer.KryoSerializer"),
    #('spark.shuffle.service.enabled', 'true'),
    #("spark.driver.maxResultSize", "5G"),
    #("spark.sql.shuffle.partitions", '12000'),
    #("spark.reducer.maxReqsInFlight", 1),
    #("spark.shuffle.io.retryWait", "60s"),
    #("spark.shuffle.io.maxRetries", 10),
    #("spark.network.timeout","240s")
])

In [6]:
sc=SparkContext(conf=spark_conf)

#start sqlcontext
sqlContext = SQLContext(sc)